In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import warnings
warnings.filterwarnings(action='ignore')

import random as python_random
np.random.seed(42)
python_random.seed(42)

In [2]:
PATH = './data/'

train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qua = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qua  = pd.read_csv(PATH+'test_quality_data.csv')

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [3]:
id_error = train_err[['user_id','errtype']].values

train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:46<00:00, 353115.60it/s]


(15000, 42)

In [4]:
problem = np.zeros(15000)

problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [5]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

id_error = test_err[['user_id','errtype']].values
test_error = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    test_error[person_idx - test_user_id_min,err - 1] += 1
test_error = test_error.reshape(test_error.shape[0],-1)
print(test_error.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:47<00:00, 346501.26it/s]

(14999, 42)


In [15]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from sklearn.metrics import *

tf.random.set_seed(42)

intra_op_parallelism_threads = 1
inter_op_parallelism_threads = 1

In [7]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(error, problem, test_size=0.25, random_state=42)

train_X.shape, test_X.shape, train_y.shape, test_y.shape

((11250, 42), (3750, 42), (11250,), (3750,))

In [19]:
model = tf.keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(error.shape)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [20]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.fit(train_X, train_y, epochs=50, batch_size=48, validation_split=0.25)
model.evaluate(test_X, test_y)

Epoch 1/50
176/176 [==============================] - 1s 2ms/step - loss: 2.0403 - val_loss: 4.0416
Epoch 2/50
176/176 [==============================] - 0s 2ms/step - loss: 0.9821 - val_loss: 3.6092
Epoch 3/50
176/176 [==============================] - 0s 2ms/step - loss: 0.7178 - val_loss: 4.6013
Epoch 4/50
176/176 [==============================] - 0s 2ms/step - loss: 0.4792 - val_loss: 3.5582
Epoch 5/50
176/176 [==============================] - 0s 2ms/step - loss: 0.6428 - val_loss: 6.1073
Epoch 6/50
176/176 [==============================] - 1s 3ms/step - loss: 0.4664 - val_loss: 8.5160
Epoch 7/50
176/176 [==============================] - 0s 3ms/step - loss: 0.4452 - val_loss: 10.8756
Epoch 8/50
176/176 [==============================] - 0s 2ms/step - loss: 0.5224 - val_loss: 14.2828
Epoch 9/50
176/176 [==============================] - 0s 3ms/step - loss: 0.3830 - val_loss: 15.7730
Epoch 10/50
176/176 [==============================] - 0s 3ms/step - loss: 0.5886 - val_loss: 18.

7.28730583190918

In [21]:
pred = model.predict_proba(test_X)
pred
print(roc_auc_score(test_y, pred))

0.7292756980855993


In [22]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.fit(error, problem, epochs=50, batch_size=48, validation_split=0.25)
sample_submssion['problem'] = model.predict_proba(test_error)

Epoch 1/50
235/235 [==============================] - 1s 2ms/step - loss: 24.8742 - val_loss: 0.5879
Epoch 2/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7758 - val_loss: 0.5864
Epoch 3/50
235/235 [==============================] - 0s 2ms/step - loss: 4.0151 - val_loss: 0.6909
Epoch 4/50
235/235 [==============================] - 0s 2ms/step - loss: 1.2925 - val_loss: 0.5420
Epoch 5/50
235/235 [==============================] - 0s 2ms/step - loss: 1.6173 - val_loss: 0.5236
Epoch 6/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7691 - val_loss: 0.5199
Epoch 7/50
235/235 [==============================] - 0s 2ms/step - loss: 0.6963 - val_loss: 0.5543
Epoch 8/50
235/235 [==============================] - 0s 2ms/step - loss: 0.5626 - val_loss: 0.5624
Epoch 9/50
235/235 [==============================] - 0s 2ms/step - loss: 0.8025 - val_loss: 0.5191
Epoch 10/50
235/235 [==============================] - 0s 2ms/step - loss: 0.5362 - val_loss: 0.523

In [23]:
sample_submssion.to_csv("./submission/submission_20210114-3.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.998689
1,30001,0.287556
2,30002,0.401343
3,30003,0.557216
4,30004,0.336178
...,...,...
14994,44994,0.315735
14995,44995,0.237339
14996,44996,0.925208
14997,44997,0.161285
